In [ ]:
import numpy as np
import pandas as pd
import fastf1
import os
import warnings
import xlsxwriter
warnings.simplefilter(action='ignore', category=FutureWarning) #ignore future warnings so our file is not polluted with unnecessary info.

In [ ]:
fastf1.Cache.enable_cache(os.getenv('LOCALAPPDATA') + "/pip/cache/fastF1")

In [ ]:
df_races = pd.DataFrame()
df_weather_data = pd.DataFrame()
df_races_control_data = pd.DataFrame()
df_drivers_info = pd.DataFrame()
df_telemetries = pd.DataFrame()

In [ ]:
for i in range(1,23):
    session = fastf1.get_session(2022, i, 'R')
    session.load()

    df_race = session.laps
    df_race['LapTime(ms)'] = df_race['LapTime']//np.timedelta64(1, 'ms')     
    df_race['EventName'] = session.event.EventName
    df_race['EventDate'] = session.event.EventDate
    df_races = pd.concat([df_races, df_race])

    session.weather_data['EventName'] = session.event.name
    df_weather_data = pd.concat([df_weather_data,session.weather_data])

    session.race_control_messages['EventName'] = session.event.name
    df_races_control_data = pd.concat([df_races_control_data, session.race_control_messages])

    for driver in session.drivers:

        df_driver_info = pd.DataFrame()
        
        for item in list(session.get_driver(driver).index):
            
            df_driver_info['EventName'] = pd.Series(session.event.EventName)       
            df_driver_info[item] = session.get_driver(driver)[item]
        
        df_drivers_info = pd.concat([df_drivers_info,df_driver_info])

        session_d = session.laps.pick_driver(driver)
        for lap in range(len(session_d)): #len(session_d)
            print("Race " + str(i) + " of 22" + " - " "Driver " + str(session.drivers.index(driver)+1) + " of "+ str(len(session.drivers)) +" - " + "Lap " + str(lap + 1) +" of " + str(len(session_d)))
            telemetry_lap = session_d.iloc[lap,].get_telemetry() 
            df_tel = telemetry_lap['RelativeDistance'].diff().groupby(telemetry_lap['nGear']).sum().reset_index()
            df_tel['DriverNumber'] = driver
            df_tel['LapNumber'] = lap + 1
            df_tel['EventName'] = session.event.EventName
            df_telemetries = pd.concat([df_telemetries, df_tel])

df_gear = df_tel.pivot(index=['DriverNumber','LapNumber','EventName'],columns = ['nGear'])
df_gear = df_gear.rename(columns={'RelativeDistance': "Gear"})
df_gear.columns = ['_'.join(str(s).strip() for s in col if s) for col in df_gear.columns]
df_gear.reset_index(inplace = True)

df_races = pd.merge(df_races,df_gear, how = 'left', on = ['DriverNumber','LapNumber','EventName'])

df_races.to_excel("Races.xlsx")
df_weather_data.to_excel("Weather.xlsx")
df_races_control_data.to_excel("Control Messages.xlsx")
df_drivers_info.to_excel("Driver Race Info.xlsx")
df_telemetries.to_excel("Telemetries.xlsx")
